# Chapter 1: Using neural nets to recognize handwritten digits

Starting with definition of artificial neur*ons*, i.e. perceptrons and sigmoid neuron.

## Perceptrons

A perceptron is a simple neuron (a node in the neural network) that takes $n$ binary inputs and returns a binary output. It usually attaches weights to each input, and if the incoming signals, multiplied by their respective weights, are higher than a given threshold, the output signal is fired.


In [1]:
import numpy as np

class perceptron:
    def fire(self, inputs, weights, threshold):
        return 1 if np.sum(inputs*weights) > threshold else 0

In [2]:
my_neuron = perceptron()
my_neuron.fire(
    inputs = np.array([1, 1]), 
    weights = np.array([0.6, 0.5]), 
    threshold = 1
)

1

By stacking such neurons in multiple layers, the neural network can make increasingly subtle decisions.

For notational simplicity that will become clear later on, we use instead of a *threshold* a *bias* term, which describes something like "the difficulty of getting this neuron to fire."

In [6]:
class perceptron:
    def fire(self, inputs, weights, bias):
        return 1 if np.dot(inputs,weights) + bias > 0 else 0

In [7]:
my_new_neuron = perceptron()
my_new_neuron.fire(
    inputs = np.array([0, 0, 1]), 
    weights = np.array([2, 2, 6]), 
    bias = -5
)

1

Perceptrons can be configured to compute logical functions, such as "and", "or", or "nand". For example, here is a "nand" perceptron:

In [80]:
class nand_perceptron:
    weights = np.array([-2, -2])
    bias = 3
    def fire(self, inputs, weights = weights, bias = bias):
        return 1 if np.dot(inputs,weights) + bias > 0 else 0

In [81]:
nand_neuron = nand_perceptron()
nand_neuron.fire(inputs = np.array([1, 0]))

1

This is pretty cool, because NAND is universal for computation. That is, we can build up any computation out of NAND gates.

In [30]:
class classic_nand_gate:
    def fire(self, inputs):
        return False if inputs[0] and inputs[1] else True

In [32]:
my_nand_gate = classic_nand_gate()
my_nand_gate.fire(inputs = [True, True])

False

For example, we can build up bitwise addition out of NAND gates. And since we can build up NAND out of a perceptron, perceptrons are universal for computation too.

In [82]:
def add_bits(bit1, bit2, type_of_gate):
    
    if type_of_gate == "classic":
        gate = classic_nand_gate()
    elif type_of_gate == "perceptron":
        gate = nand_perceptron()
    
    gate0_in = [bit1, bit2]
    gate0_out = gate.fire(inputs = gate0_in)
    
    gate1_in = [bit1, gate0_out]
    gate1_out = gate.fire(inputs = gate1_in)
    
    gate2_in = [gate0_out, bit2]
    gate2_out = gate.fire(inputs = gate2_in)
    
    gate3_in = [gate1_out, gate2_out]
    gate3_out = gate.fire(inputs = gate3_in)
    
    gate4_in = [gate0_out, gate0_out]
    gate4_out = gate.fire(inputs = gate4_in)
    
    bit_sum = gate3_out
    carry_bit = gate4_out
    
    return str(int(carry_bit)) + str(int(bit_sum))

In [79]:
assert add_bits(1, 1, type_of_gate="classic") == '10'
assert add_bits(0, 1, type_of_gate="classic") == '01'
assert add_bits(1, 0, type_of_gate="classic") == '01'
assert add_bits(0, 0, type_of_gate="classic") == '00'

assert add_bits(1, 1, type_of_gate="perceptron") == '10'
assert add_bits(0, 1, type_of_gate="perceptron") == '01'
assert add_bits(1, 0, type_of_gate="perceptron") == '01'
assert add_bits(0, 0, type_of_gate="perceptron") == '00'

This isn't so exciting on its own. BUT, because perceptrons have weights and bias values that can be tuned to *configure* lower-level logical functions that can in turn be combined to configure higher-level computations, we should be able to expose an artificial neural network to an external source of data that will tune those weights and biases for us. We can then end up with a neural network that is trained to represent any arbitrary computation.